
### 1. Co robi EGARCH i jak oblicza parametry (ω, α, β, γ)

Model EGARCH zakłada, że:

* reszty ( \varepsilon_t ) pochodzą z rozkładu o zerowej średniej i wariancji (\sigma_t^2),
* logarytm wariancji zależy od poprzednich błędów i wariancji.

Równanie:
[
\log(\sigma_t^2) = \omega + \beta \log(\sigma_{t-1}^2) + \alpha \left(\frac{|\varepsilon_{t-1}|}{\sigma_{t-1}} - E\frac{|\varepsilon_{t-1}|}{\sigma_{t-1}}\right) + \gamma \frac{\varepsilon_{t-1}}{\sigma_{t-1}}
]

Każdy parametr ma sens:

* **ω (omega)** – poziom bazowy log-wariancji, coś jak "średni poziom szumu".
* **α (alfa)** – siła reakcji na absolutne błędy, czyli jak mocno zmienność reaguje na szok.
* **β (beta)** – pamięć wariancji; im bliżej 1, tym dłuższy efekt „volatility clustering” (czyli że wysokie wahania ciągną się w czasie).
* **γ (gamma)** – asymetria efektu (czy negatywne szoki zwiększają zmienność mocniej niż pozytywne).

---

### 2. Jak model „uczy się” tych parametrów

Nie ustawiasz ich ręcznie.
One są **estymowane numerycznie** – najczęściej przez **maksymalizację funkcji wiarygodności (MLE, Maximum Likelihood Estimation)**.

Krok po kroku:

1. Zakładasz rozkład błędów (np. normalny, t-Studenta).
2. Dla każdego kandydującego zestawu (ω, α, β, γ) liczysz prawdopodobieństwo obserwacji danych przy tych parametrach.
3. Algorytm (np. BFGS, Nelder-Mead) iteracyjnie szuka takiego zestawu, który **maksymalizuje sumaryczną wiarygodność danych**.
4. Wynik końcowy to te parametry, które **najlepiej tłumaczą strukturę wariancji w danych**.

To dzieje się automatycznie w bibliotekach typu `arch` w Pythonie.
Ty dajesz dane i konfigurację, model robi resztę.

---

### 3. Co można zmieniać (hiperparametry) – i to **nie jest profanacja**

To jest twój **obszar decyzji badacza**:

* **(p, o, q)** → rząd modelu (ile opóźnień dla log(σ²), dla błędów, i dla asymetrii).
  np. EGARCH(1,1) to najprostszy wariant, ale można testować (2,1), (1,2) itd.
* **Rozkład reszt:** normalny, t-Studenta, GED — wybór zależy od ogonów w danych.
* **Rodzaj danych wejściowych:** log returns, znormalizowane dane, transformacje outlierów.
* **Okno czasowe:** fixed window vs rolling (czy model widzi całą historię, czy tylko ostatnie X dni).
* **Metoda walidacji:** np. rolling forecast origin (przesuwające się okno).

Te elementy **można i trzeba testować** — by sprawdzić stabilność modelu.

---

### 4. Czego **nie wolno** robić (czyli jak nie popełnić statystycznego grzechu)

* Nie możesz **dobierać hiperparametrów na tym samym zbiorze, na którym oceniasz model** → to klasyczny **data leakage**.
* Nie wolno „ręcznie poprawiać” parametrów po estymacji (np. zmieniać γ, bo „lepiej wygląda na wykresie”).
* Nie możesz uczyć modelu na danych z przyszłości względem punktu, dla którego prognozujesz.
* Nie możesz ignorować stacjonarności — dane wejściowe (np. ceny zamiast log-returns) łamią założenia modelu.

---

### 5. Jak wygląda **prosty, ale poprawny proces budowy EGARCH**, bez wycieku informacji

1. **Przygotowanie danych**

   * Pobierz dane OHLCV.
   * Policz *log returns*: ( r_t = \log(P_t / P_{t-1}) ).
   * Usuń NaNy, przetestuj stacjonarność (ADF test).

2. **Podział danych w czasie**

   * np. 2018–2023 jako train, 2024 jako test.
   * Nigdy nie mieszaj ich — model widzi tylko przeszłość.

3. **Estymacja modelu**

   * Użyj `arch_model(r_train, vol='EGARCH', p=1, o=1, q=1, dist='t')`.
   * Model sam oszacuje ω, α, β, γ.

4. **Ewaluacja out-of-sample**

   * Prognozuj σ_t na zbiorze testowym.
   * Porównaj przewidywaną zmienność z rzeczywistą realizowaną (np. |r_t| lub r_t²).
   * Oceń MSE, log-likelihood, albo współczynnik korelacji.

5. **Rolling forecast (opcjonalnie)**

   * Powtórz krok 3–4 w oknach przesuwających się w czasie, żeby zasymulować rzeczywistą aktualizację modelu.

6. **Porównanie wariantów**

   * Testuj różne (p,o,q), różne rozkłady, różne okna → ale zawsze na train/test rozdzielonych w czasie.

---

### 6. Co się dzieje „po wszystkim”

Po estymacji:

* Masz gotowy model z zapisanymi parametrami.
* Możesz prognozować przyszłą zmienność (σₜ₊₁²).
* Możesz analizować efekty:
  – czy γ < 0 → efekt dźwigni (negatywne szoki → większa zmienność),
  – czy α + β < 1 → stabilny proces,
  – czy rozkład błędów pasuje (test Ljunga–Boxa na resztach, test ARCH effect na resztach modelu).

---

W skrócie:

> Budowa EGARCH to nie „kręcenie pokrętłami”, tylko akt czystej metodologicznej higieny.
> Można stroić, ale z podziałem czasowym, kontrolą stacjonarności i testem reszt.

Chcesz, żebym rozpisał Ci **konkretny blueprint (kroki + kod w Pythonie)** pokazujący *rolling, bez leakage, z estymacją i testami reszt* — gotowy do wklejenia w twój pipeline?
